My first Jupyter notebook to run a Python script that will parse the complete works of Shakespeare into JSONL documents as preparation for fine-tuning GPT-3
First - we need to install the [transformers package](https://huggingface.co/transformers/installation.html?highlight=transformers)

In [ ]:
pip install transformers

Next we will new-up an instance of the GPT2TokenizerFast class as this gives us a utility for tokenizing input text (words and partial words, including individual whitespace characters [at times] get transformed into tokens).  Tokens are simple numeric values that the GPT-2 and GPT-3 created when parsing all their 1.5 Billion parameters over their 8 million web pages.  While I don't claim to understand all the math, my rudimentary understanding is that a Machine Learning model does lightning-quick statistical calculations on the most likely words to "come next" when presented with enough input data.  So if you take the entire prologue of the US Constitution, and then you leave off the last sentence, with enough training data, the ML model GPT-3 will predict the final sentences to complete the Constitution correctly.  It does this by taking the words, tokenizing them to their numeric ids, then crunching all those numbers up together and statistically saying "the correct next words that are highest probability are these tokens" and spits out the ids of the tokens, which are then transformed by the model back into words, and built into a string, and that string is sent back to the user via the API.  

With Fine Tuning - what we're trying to do is lean on the scales of that statistical model just a bit and give it a bias.  By pushing in a bunch of "documents" (which are basically just JSON objects where there is a prompt and a completion, and the JSONL format is like a big CSV of JSON objects - I don't really understand the need for the "L" file extension unless it's just a "list of JSON" which - though I haven't looked it up - is likely) we can lean on those proverbial statistical scales and push them a bit in our favor.  The GPT-3 model was training on 1.5B parameters over 8 million webpages, so it would take a LOT of input data in order to *heavily* influence the outcome of a completion, but we can nudge things a bit in our favor.

I've used the complete works of Shakespeare here as a test - I need a clean data set, I need a well-defined tone of voice, and I don't want to spend a ton of time making it all up from scratch or trying to find things that match my ultimate goal.  

My hypothesis here is that if we can fine tune the GPT-3 model, and I ask a question, I hope to get some "forsooth" type stuff in the replies - something very Shakesperean - which can prove that my fine tuning had some influence on the outcome.  If I can achieve that objective, it will give me confidence that we can influence the model with the data that we want for a personality, even though the personality that we come up with may be very "modern" sounding and therefore if we jump straight to fine tuning with a "modern" voice, we may not be able to detect the influence in the completion.  I kind of want to "break" it right now, and force it to return very Old English-y responses.  We're exaggerating the outcome on purpose to prove that something happened.

In [28]:
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
import os
import json

This will be the main part of the script - we start with a simple test to prove that the tokens are being generated and counted, and then we will feed in all the documents and chunk them up into JSONL files.

First we'll just leave in the example given us by Huggingface to demonstrate that a string turns into a dictionary of ids where each id is a token.

In [2]:
ids = tokenizer("Hello world")['input_ids']
print('Count of tokens {}'.format(len(ids)))

Count of tokens 2


Look in the "source_files" directory and loop through each file, going for a maximum of 2048 "tokens" - we cannot know how many tokens each chunk of data might contain exactly, so the GPT-3 guidance is that 100 tokens is roughly 75 words (it's not an exact science), so with this guidance, we will aim for chunks of text that is 70% of 2048 or 1433.6 / 1434 words per JSONL completion.  It will probably be inefficient in Python to add the words one-by-one and count after each (even though Python is fast), so we'll go line-by-line in the source materials and then count, and if we tip the scales at 1434 we will batch to the next.  This means we'll probably end up with some JSONL documents with a completion of larger than 1434 words, but as we've gone for 70% instead of 75% of the 2048 token target, we should hopefully have a small buffer.  We want each document to be less than 2048 tokens, but only just.  The best outcome will be 2048 tokens bang-on, but more like we'll end up with like 2020s or 2030s which is still going to be ok for this test.

python's readline has a *sizehint* parameter that will read a file up until the declared number of bytes.  An online page I found says that basically 1 word is 2 bytes, and we want 1434 words, so we're looking for 2868 byte chunks from the file.  

After writing the above, and reading more on Python handling files, I determined that the size of the files we are dealing with is tiny, and so there is no reason to get fancy with chunking the file into smaller pieces.  We are not going to run into memory problems loading a 1 MB text file into memory.  So just read the whole file into memory and then perform operations.  If you were to use multi-GB files, then you would need to pay attention to chunking and memory consumption.

In [50]:
# Whoops - saved the ipynb in the source_files dir instead of the project root - handy to check where you actually are!
# print(os.getcwd())

def divide_chunks(l, n):
    # looping till length l
    for i in range(0, len(l), n): 
        yield l[i:i + n]

def parse_play_into_jsonl(play):
    no_newline = ""
    play = "source_files/{}".format(play)
    with open(play) as fp:
        Lines = fp.readlines()
        for line in Lines:
            # The data has \n at the end of each line, we get rid of that but add a single space to turn the play into prose
            no_newline += line.rstrip()+" "
            play_as_list = no_newline.split()

    # 1434 being our target number of words to stay at 70% of the 2048 token value (approximate)
    chunked = list(divide_chunks(play_as_list, 1434))
    for e in chunked:
        line_string = ""
        for w in e:
            # A dictionary is just the words without spaces, but reconstructing the string we need spaces between words
            line_string += w+" "
        jsonl_element = {"prompt": "", "completion": line_string.rstrip()}
        with open('source_files/shakespeare_json.jsonl', 'a') as f:
            # Writes the JSON object to a single line in the file, then inserts a line return.  This is the JSONL format expected by OpenAI
            json.dump(jsonl_element, f, ensure_ascii=False)
            f.write('\n')

for filename in os.listdir('source_files/'):
    if filename.endswith(".txt"):
        parse_play_into_jsonl(filename)
        # Trying a single file from the directory below:
        # parse_play_into_jsonl("a_lovers_complaint.txt")


From the command line, OpenAI gives us a CLI utility to build and test our model - instructions are available within their documentation, but I'll include them here for reference.  You will need to perform the openai login step which requires your API key.


`openai tools fine_tunes.prepare_data -f /Users/tyler/Software_Development/gpt3-text-preparer-python/source_files/shakespeare_json.jsonl`

`openai api fine_tunes.create -t "/Users/tyler/Software_Development/gpt3-text-preparer-python/source_files/shakespeare_json_prepared.jsonl"`
    
    Upload progress: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5.03M/5.03M [00:02<00:00, 2.35Mit/s]
    Uploaded file from /Users/tyler/Software_Development/gpt3-text-preparer-python/source_files/shakespeare_json_prepared.jsonl: file-lBEGoHcemdd50BwTGPI7oYYy
    Created fine-tune: ft-R4lOiZemqAUy8LluGj6ldlp0
    Streaming events until fine-tuning is complete...

    (Ctrl-C will interrupt the stream, but not cancel the fine-tune)
    [2021-07-28 13:12:11] Created fine-tune: ft-R4lOiZemqAUy8LluGj6ldlp0
    [2021-07-28 13:12:14] Fine-tune enqueued. Queue number: 0
    [2021-07-28 13:12:18] Fine-tune started
    [2021-07-28 13:16:19] Completed epoch 1/4
    [2021-07-28 13:18:42] Completed epoch 2/4
    [2021-07-28 13:21:05] Completed epoch 3/4
    [2021-07-28 13:23:29] Completed epoch 4/4
    [2021-07-28 13:24:28] Uploaded model: curie:ft-uneeq-2021-07-28-18-24-26
    [2021-07-28 13:24:31] Uploaded result file: file-ZOBCNNmS1JxonhKCU9RAhxZs
    [2021-07-28 13:24:32] Fine-tune succeeded

    Job complete! Status: succeeded 🎉
    Try out your fine-tuned model:

    openai api completions.create -m curie:ft-uneeq-2021-07-28-18-24-26 -p <YOUR_PROMPT>

`openai api completions.create -m curie:ft-uneeq-2021-07-28-18-24-26 --stop "[\"\n\n###\n\n\",\"%\",\"\n Human:\",\"\n AI:\"]" -M 64 -t 0.9 -n 1 -p "Human: Do you think I should go cycling?\n AI: Yes, that is a very good idea.  Do you often go cycling?\n Human: Yes, I really enjoy the exercise and being outside.\n AI:\n\n###\n\n"`